# Load optuna library

In [2]:
import numpy as np
import pandas as pd
import os, sys 
import optuna

# Load the dataset and split

In [3]:
from HELPpy.preprocess.loaders import load_features
from HELPpy.utility.utils import pandas_readcsv
from sklearn.model_selection import train_test_split
path = '../../data'
tissue = 'Kidney'
attributes = load_features([os.path.join(path, f'{tissue}_BIO.csv'), 
                            os.path.join(path, f'{tissue}_CCcfs.csv'),
                            os.path.join(path, f'{tissue}_EmbN2V_128.csv')
                           ], 
                            fixnans=[True, True, False], normalizes=['std', 'std', None], verbose=False, show_progress=True)
labelnme = f'{tissue}_HELP.csv'
label = pandas_readcsv(os.path.join(path,labelnme), descr=f'{labelnme}', index_col=0).replace({'E': 1, 'aE':0, 'sNE': 0})
idx_common = np.intersect1d(attributes.index.values, label.index.values)
attributes = attributes.loc[idx_common]
label = label.loc[idx_common]
#X_train, X_test, y_train, y_test = train_test_split(attributes, label, shuffle=False)
#train = pd.concat([X_train, y_train], axis=1)
#test = pd.concat([X_test, y_test], axis=1)

Kidney_BIO.csv:   0%|          | 0/19293 [00:00<?, ?it/s]

Kidney_EmbN2V_128.csv:   0%|          | 0/19314 [00:00<?, ?it/s]

Kidney_HELP.csv:   0%|          | 0/17829 [00:00<?, ?it/s]

/var/folders/vh/63gp320s2kg04slqwz_wx8yc0000gn/T/ipykernel_1912/92753867.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  label = pandas_readcsv(os.path.join(path,labelnme), descr=f'{labelnme}', index_col=0).replace({'E': 1, 'aE':0, 'sNE': 0})


# Start tuning session

In [6]:
tissue = 'Kidney'
from sklearn.metrics import *
from HELPpy.models.prediction import VotingEnsembleLGBM
from optuna import Trial

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import balanced_accuracy_score, make_scorer
from HELPpy.models.prediction import VotingEnsembleLGBM
def objective_cv(trial : Trial, X : pd.DataFrame, y : np.ndarray | pd.Series, random_state : int=42):
    params = {
        'learning_rate': trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        'n_voters': trial.suggest_int('n_voters', 1, 20, step=1),
        'n_estimators': trial.suggest_int('n_estimators', 60, 200, step=20),
        'boosting_type': trial.suggest_categorical("boosting_type", ["gbdt", "dart"])
    }
    gbm = VotingEnsembleLGBM(**params)
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    ba_scorer = make_scorer(balanced_accuracy_score)
    scores = cross_val_score(gbm, X, y, scoring=ba_scorer, cv=kf)
    #return np.min([np.mean(scores), np.median([scores])])
    return np.mean(scores)

#def objective(trial):
#    params = {
#        'learning_rate': trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
#        'n_voters': trial.suggest_int('n_voters', 1, 20, step=1),
#        'n_estimators': trial.suggest_int('n_estimators', 60, 200, 20),
#        'boosting_type': trial.suggest_categorical("boosting_type", ["gbdt", "dart"])
#    }
#    gbm = VotingEnsembleLGBM(**params)
#    gbm.fit(X_train, y_train)
#    preds = gbm.predict(X_test)
#    ba = balanced_accuracy_score(y_test, preds)
    #mcc = matthews_corrcoef(y_test, preds)
#    return ba#, mcc

savepath = '../../results'
study = optuna.create_study(
    study_name=f'velgbm_{tissue}',
    #directions=["minimize", "maximize"],
    direction='maximize',
    load_if_exists=True,
    storage=f'sqlite:///{savepath}/veLGBM_{tissue}_ba_cv.db'
    )
study.optimize(lambda trial: objective_cv(trial, attributes, label), n_trials=50)
best_params = study.best_params
print('Best Params:', best_params)
df = study.trials_dataframe()
df.to_csv(os.path.join(f'velgbm_{tissue}_hypersearch_ba_cv.csv'))
df.sort_values('value', ascending=False)

[I 2024-06-04 16:13:04,422] A new study created in memory with name: velgbm_Kidney
/var/folders/vh/63gp320s2kg04slqwz_wx8yc0000gn/T/ipykernel_1912/2795621893.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform("learning_rate", 1e-3, 0.1),
/var/folders/vh/63gp320s2kg04slqwz_wx8yc0000gn/T/ipykernel_1912/2795621893.py:7: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'n_voters': trial.suggest_int('n_voters', 1, 20, 1),
[I 2024-06-04 16:13:47,941] Trial 0 finished with value: 0.8406585425586718 and parameters: {'learning_rate': 0.0688291506446236, 'n_voters': 10, 'boosting_type': 'dart'}. Best is trial 0 with value: 0.8406585425586718.
/var/folders/vh/63gp320s2kg04slqwz_wx8yc0000gn/T/ipykernel_1912/2795621893.py:6

   number     value             datetime_start          datetime_complete  \
0       0  0.840659 2024-06-04 16:13:04.423539 2024-06-04 16:13:47.940819   
1       1  0.840187 2024-06-04 16:13:47.943669 2024-06-04 16:14:33.709639   
2       2  0.500000 2024-06-04 16:14:33.710131 2024-06-04 16:15:05.624432   
3       3  0.738861 2024-06-04 16:15:05.624908 2024-06-04 16:16:00.426407   
4       4  0.820660 2024-06-04 16:16:00.426874 2024-06-04 16:17:03.621266   

                duration params_boosting_type  params_learning_rate  \
0 0 days 00:00:43.517280                 dart              0.068829   
1 0 days 00:00:45.765970                 dart              0.082820   
2 0 days 00:00:31.914301                 dart              0.002568   
3 0 days 00:00:54.801499                 dart              0.001352   
4 0 days 00:01:03.194392                 dart              0.008155   

   params_n_voters     state  
0               10  COMPLETE  
1               11  COMPLETE  
2                

# Test the best model

In [1]:
from HELPpy.models.prediction import VotingEnsembleLGBM, k_fold_cv
clf = VotingEnsembleLGBM(**study.best_params)
# train using create_model
df_scores, scores, predictions = k_fold_cv(attributes, label, clf, n_splits=5, seed=0, show_progress=True, verbose=True)
df_scores

NameError: name 'study' is not defined

In [27]:
#df = pd.read_csv(f'velgbm_{tissue}_hypersearch_mcc_ba_2.csv').sort_values(['values_1', 'values_0'], ascending=[False, False]).rename(columns={'values_0': 'BA', 'values_1': 'MCC'})
df = pd.read_csv(f'velgbm_Kidney_hypersearch_ba_cv.csv').sort_values('value', ascending=False).rename(columns={'value': 'BA'})
df.columns = df.columns.str.replace(r'params_', '')
selcolumns = ['boosting_type',	'learning_rate',	'n_estimators',	'n_voters', 'BA']
stds = df.std(numeric_only=True)
#df['BA+MCC'] = df.apply(lambda x: x.BA * stds.loc['BA'] + x.MCC * stds.loc['MCC'], axis=1)
#df['BA+MCC'] = df.apply(lambda x: x.BA  + x.MCC , axis=1)
print(df[selcolumns].sort_values('BA', ascending=False).to_latex())

\begin{tabular}{llrrrr}
\toprule
 & boosting_type & learning_rate & n_estimators & n_voters & BA \\
\midrule
37 & gbdt & 0.094505 & 200 & 13 & 0.893151 \\
15 & gbdt & 0.098300 & 140 & 10 & 0.891459 \\
44 & gbdt & 0.076452 & 200 & 12 & 0.890954 \\
43 & gbdt & 0.075168 & 200 & 12 & 0.890826 \\
41 & gbdt & 0.078591 & 200 & 13 & 0.890602 \\
33 & gbdt & 0.098020 & 180 & 13 & 0.890241 \\
31 & gbdt & 0.059095 & 160 & 11 & 0.889936 \\
34 & gbdt & 0.085756 & 200 & 13 & 0.889739 \\
22 & gbdt & 0.063759 & 180 & 9 & 0.889298 \\
30 & gbdt & 0.054934 & 160 & 12 & 0.889146 \\
36 & gbdt & 0.076796 & 200 & 14 & 0.889028 \\
23 & gbdt & 0.065602 & 160 & 9 & 0.888994 \\
42 & gbdt & 0.076634 & 200 & 16 & 0.888759 \\
40 & gbdt & 0.044127 & 180 & 10 & 0.888419 \\
4 & gbdt & 0.088891 & 140 & 15 & 0.888175 \\
39 & gbdt & 0.098960 & 140 & 14 & 0.887998 \\
49 & gbdt & 0.057674 & 200 & 16 & 0.887826 \\
11 & gbdt & 0.059871 & 180 & 15 & 0.886745 \\
47 & gbdt & 0.049777 & 200 & 14 & 0.886566 \\
29 & gbdt & 0.042902

# Load exisiting study from DB

In [12]:
import os
os.listdir("/Users/maurizio/HELP/results")

['EvsAEKidney.csv',
 'EvsNEKidney.csv',
 'HELP_AEvsSNE_Kidney.png',
 'HELP_EvsNE_Lung.png',
 'veLGBM_Kidney_ba_cv_min_mean_med.db',
 'AEvsSNEKidney.csv',
 'EvsSNEKidney.csv',
 'HELP_EvsSNE_Kidney.png',
 'EvsAELung.csv',
 'HELP_EvsNE_Kidney.png',
 'EvsSNELung.csv',
 'EvsNELung.csv',
 'HELP_EvsAE_Kidney.png',
 'HELP_EvsAE_Lung.png',
 'HELP_AEvsSNE_Lung.png',
 'veLGBM_Kidney_ba_cv.db',
 'HELP_EvsSNE_Lung.png',
 'AEvsSNELung.csv']

In [18]:
%pwd

'/Users/maurizio/HELP/HELPpy/notebooks'

In [19]:
import optuna
study = optuna.load_study(study_name='velgbm_Kidney', storage="sqlite:///../../results/veLGBM_Kidney_ba_cv.db")

RuntimeError: The runtime optuna version 3.6.1 is no longer compatible with the table schema (set up by optuna 3.1.1). Please execute `$ optuna storage upgrade --storage $STORAGE_URL` for upgrading the storage.

In [24]:
import pandas as pd
print(pd.read_csv("velgbm_Kidney_hypersearch_ba_cv.csv").replace({'':'','':'','':'','':''})sort_values('value', ascending=False).to_latex())

\begin{tabular}{lrrrllllrrrl}
\toprule
 & Unnamed: 0 & number & value & datetime_start & datetime_complete & duration & params_boosting_type & params_learning_rate & params_n_estimators & params_n_voters & state \\
\midrule
37 & 37 & 37 & 0.893151 & 2024-06-05 11:48:54.638670 & 2024-06-05 11:50:51.129284 & 0 days 00:01:56.490614 & gbdt & 0.094505 & 200 & 13 & COMPLETE \\
15 & 15 & 15 & 0.891459 & 2024-06-05 11:16:36.759285 & 2024-06-05 11:17:48.725832 & 0 days 00:01:11.966547 & gbdt & 0.098300 & 140 & 10 & COMPLETE \\
44 & 44 & 44 & 0.890954 & 2024-06-05 12:01:09.467990 & 2024-06-05 12:03:01.279431 & 0 days 00:01:51.811441 & gbdt & 0.076452 & 200 & 12 & COMPLETE \\
43 & 43 & 43 & 0.890826 & 2024-06-05 11:59:18.529245 & 2024-06-05 12:01:09.439684 & 0 days 00:01:50.910439 & gbdt & 0.075168 & 200 & 12 & COMPLETE \\
41 & 41 & 41 & 0.890602 & 2024-06-05 11:55:19.637376 & 2024-06-05 11:57:14.427060 & 0 days 00:01:54.789684 & gbdt & 0.078591 & 200 & 13 & COMPLETE \\
33 & 33 & 33 & 0.890241 & 